In [ ]:
!pip install crewai google-generativeai "crewai[tools]"


In [ ]:
# Get API keys from environment variables
from google.colab import userdata
import google.generativeai as genai
import os

GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")
if not GEMINI_API_KEY:
    print("Warning: GEMINI_API_KEY not found. Gemini model will not run.")

genai.configure(api_key=GEMINI_API_KEY)

In [ ]:
from crewai import Agent, Task, Crew, LLM, Process
from google.generativeai import GenerativeModel

gemini_model = "gemini-2.5-flash-lite"
gemini_model = "gemini-2.5-pro"
gemini_model = "gemini-2.5-flash"
llm = LLM(
    model=gemini_model,
    temperature=0.1,
    api_key=GEMINI_API_KEY
)


Input:

Topic (e.g. “GenAI in education”)

Duration (e.g. 30 days)

Platforms (e.g. Blog, YouTube, LinkedIn)

Output:

Content pillars / subtopics

SEO-optimized titles & hooks

A simple content calendar (day-by-day or post-by-post)

## Architecture
### Agents

1. TopicResearcherAgent: Breaks the main topic into 4–8 subtopics / content pillars.

2. SEOStrategistAgent: Suggests catchy titles, hooks, and basic keyword ideas for each subtopic.

3. ContentPlannerAgent: Turns it into a concrete content calendar:

        Which day

        What platform

        What type (short, long-form, carousel, etc.)

        We’ll wire them into a sequential crew:

        User brief → Research → SEO ideas → Final calendar

In [ ]:
from crewai import Agent

topic_researcher = Agent(
    role="Topic Researcher",
    goal=(
        "Break a broad topic into clear content pillars and subtopics that are "
        "interesting and useful for the target audience."
    ),
    backstory=(
        "You are a content strategist who understands audiences and can identify "
        "the most valuable angles for a topic."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

seo_strategist = Agent(
    role="SEO & Hook Strategist",
    goal=(
        "For each content pillar, propose SEO-friendly titles, hooks, and basic "
        "keyword ideas that can perform well on search and social platforms."
    ),
    backstory=(
        "You are an SEO and social media expert, skilled at crafting catchy titles "
        "and identifying search intent."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)

content_planner = Agent(
    role="Content Planner",
    goal=(
        "Create a concrete content calendar over a chosen timeframe, assigning "
        "specific pieces of content to specific days and platforms."
    ),
    backstory=(
        "You are a content operations manager who turns strategy and ideas into a "
        "realistic publishing schedule."
    ),
    llm=llm,
    verbose=True,
    allow_delegation=False,
)


In [ ]:
from crewai import Task

def build_content_planner_tasks(main_topic: str, days: int, platforms: list):
    platforms_str = ", ".join(platforms)

    # Task 1: Research pillars & subtopics
    task_research = Task(
        description=(
            "A creator wants to produce consistent content on the following topic:\n\n"
            f"TOPIC: {main_topic}\n\n"
            "Your job:\n"
            "1. Identify 4 to 8 major content pillars or themes.\n"
            "2. For each pillar, list 3–5 specific subtopics or angles.\n"
            "3. Focus on what would be most helpful or interesting for the target audience.\n\n"
            "Return the result in a clear, structured format with headings."
        ),
        expected_output=(
            "A structured list of content pillars with subtopics for the given main topic."
        ),
        agent=topic_researcher,
    )

    # Task 2: SEO + Hooks for each pillar/subtopic
    task_seo = Task(
        description=(
            "You will receive content pillars and subtopics for a main topic.\n"
            "For EACH subtopic, propose:\n"
            "- 1–2 SEO-friendly titles\n"
            "- 1 strong hook or opening line\n"
            "- 2–3 keyword ideas (short phrases)\n\n"
            "Make the titles and hooks engaging but not clickbait.\n"
            "Use the latest context with the pillars and subtopics provided earlier."
        ),
        expected_output=(
            "For each subtopic: titles, hooks, and keyword ideas in a structured format."
        ),
        agent=seo_strategist,
    )

    # Task 3: Turn everything into a calendar
    task_calendar = Task(
        description=(
            "Using the pillars, subtopics, and SEO ideas generated previously, "
            "create a concrete content calendar.\n\n"
            f"Constraints:\n- Timeframe: {days} days\n- Platforms: {platforms_str}\n"
            "- Aim for 1–2 pieces of content per day.\n"
            "- Mix different pillars across the days.\n"
            "- For each content item, specify:\n"
            "  * Day number (e.g., Day 1, Day 2, ...)\n"
            "  * Platform\n"
            "  * Content type (e.g., YouTube video, blog post, LinkedIn post, short)\n"
            "  * Title\n"
            "  * Short description of the angle or hook.\n\n"
            "Return the output as a clearly structured list or table-like text."
        ),
        expected_output=(
            "A day-by-day content calendar with platform, content type, title, and short description."
        ),
        agent=content_planner,
    )

    return [task_research, task_seo, task_calendar]


In [ ]:
from crewai import Crew, Process

def generate_content_plan(main_topic: str, days: int = 30, platforms=None):
    if platforms is None:
        platforms = ["YouTube", "Blog", "LinkedIn"]

    tasks = build_content_planner_tasks(main_topic, days, platforms)

    crew = Crew(
        agents=[topic_researcher, seo_strategist, content_planner],
        tasks=tasks,
        process=Process.sequential,  # tasks run in order
        verbose=True,
    )

    result = crew.kickoff()
    return result

# 🔁 Example: Run for a specific topic
plan = generate_content_plan(
    main_topic="Generative AI for software developers",
    days=14,
    platforms=["YouTube", "Blog", "X (Twitter)"]
)

print("\n================ FINAL CONTENT PLAN ================\n")
print(plan)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 644856a9-0984-4486-b661-351ccbe9b01b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Topic Researcher                                                                                        │
│                                                                                                                 │
│  Task: A creator wants to produce consistent content on the following topic:                                    │
│                                                                                                                 │
│  TOPIC: Generative AI for software developers                                                                   │
│                                                                                                                 │
│  Your job:                                                                                                      │
│  1. Identify 4 to 8 major content pillars or themes.                                                            │
│  2. For each pillar, list 3–5 specific subtopics or angles.                                                     │
│  3. Focus on what would be most helpful or interesting for the target audience.                                 │
│                                                                                                                 │
│  Return the result in a clear, structured format with headings.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Topic Researcher                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here are the content pillars and subtopics for "Generative AI for software developers," designed to be         │
│  helpful and interesting for the target audience:                                                               │
│                                                                                                                 │
│  ### Content Pillars for Generative AI for Software Developers                                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **Pillar 1: Understanding Generative AI Fundamentals for Developers**                                     │
│  This pillar focuses on demystifying Generative AI, providing developers with the foundational knowledge        │
│  needed to understand its core concepts and relevance to their work.                                            │
│                                                                                                                 │
│  *   **Subtopics:**                                                                                             │
│      *   What is Generative AI? (LLMs, Diffusion Models, Transformers explained simply)                         │
│      *   Why Generative AI is a game-changer for software development (productivity, innovation, new            │
│  paradigms)                                                                                                     │
│      *   Key concepts: Tokens, embeddings, attention mechanisms, and their role in code generation              │
│      *   The evolution of AI in software development: From static analysis to intelligent assistants            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  #### **Pillar 2: Essential Tools & Platforms for AI-Powered Development**                                      │
│  This pillar explores the practical tools and platforms developers can leverage today, offering guides and      │
│  comparisons to help them choose and utilize the best options.                                                  │
│                                                                                                                 │
│  *   **Subtopics:**                                                                                             │
│      *   Deep dive into GitHub Copilot, GitLab Duo, and other AI code assistants (features, best practices,     │
│  limitations)                                                                                                   │
│      *   Leveraging LLM APIs: OpenAI (GPT), Anthropic (Claude), Google (Gemini) for custom applications         │
│      *   Exploring open-source models and platforms: Hugging Face, Llama 2, Mistral for local development       │
│      *   Cloud AI services for developers: AWS Bedrock,

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: SEO & Hook Strategist                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Pillar 1: Understanding Generative AI Fundamentals for Developers                                          │
│                                                                                                                 │
│  *   **Subtopic: What is Generative AI? (LLMs, Diffusion Models, Transformers explained simply)**               │
│      *   **Titles:**                                                                                            │
│          *   Generative AI Explained: A Developer's Guide to LLMs, Diffusion, & Transformers                    │
│          *   Demystifying Generative AI: Core Concepts for Software Developers                                  │
│      *   **Hook:** Ever wondered what's truly powering the next generation of intelligent software? Let's       │
│  break down Generative AI.                                                                                      │
│      *   **Keyword Ideas:** Generative AI explained, LLM for developers, Diffusion models simple, Transformers  │
│  AI                                                                                                             │
│                                                                                                                 │
│  *   **Subtopic: Why Generative AI is a game-changer for software development (productivity, innovation, new    │
│  paradigms)**                                                                                                   │
│      *   **Titles:**                                                                                            │
│          *   Generative AI: The Game-Changer for Software Development Productivity                              │
│          *   How Generative AI is Revolutionizing Software Development                                          │
│      *   **Hook:** Forget incremental improvements – Generative AI isn't just changing how you code, it's       │
│  redefining what's possible.                                                                                    │
│      *   **Keyword Ideas:** Generative AI productivity, AI in software development, AI innovation dev, new      │
│  coding paradigms                                                                                               │
│                                                                                                                 │
│  *   **Subtopic: Key concepts: Tokens, embeddings, attention mechanisms, and their role in code generation**    │
│      *   **Titles:**                                                                                            │
│          *   Tokens, Embeddings, Attention: Generative AI Concepts for Code Generation                          │
│          *   Understanding Generative AI's Core: Tokens, Embeddings, and Attention for Developers               │
│      *   **Hook:** Dive deep into the building blocks of AI that turn natural language into functional code.    │
│      *   **Keyword Ideas:** Tokens in AI, embeddings for code, attention mechanisms explained, AI code          │
│  generation concepts                                                                                            │
│                                                                                                                 │
│  *   **Subtopic: The evolution of AI in software develo

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7349afd2-d4f6-4f77-bbfb-895b45e555c7                                                                     │
│  Agent: SEO & Hook Strategist                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Here is a 14-day content calendar for "Generative AI for Software Developers," incorporating the provided      │
│  pillars, subtopics, and SEO ideas across YouTube, Blog, and X (Twitter).                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  **Content Calendar: Generative AI for Software Developers (14 Days)**                                          │
│                                                                                                                 │
│  | Day | Platform | Content Type | Title | Short Description / Angle | Pillar |                                 │
│  | :-- | :------- | :----------- | :---- | :------------------------ | :----- |                                 │
│  | **Day 1** | YouTube | Video | Generative AI Explained: A Developer's Guide to LLMs, Diffusion, &             │
│  Transformers | Ever wondered what's truly powering the next generation of intelligent software? Let's break    │
│  down Generative AI. | Pillar 1 |                                                                               │
│  | **Day 2** | Blog | Blog Post | Generative AI: The Game-Changer for Software Development Productivity |       │
│  Forget incremental improvements – Generative AI isn't just changing how you code, it's redefining what's       │
│  possible. | Pillar 1 |                                                                                         │
│  | | X | Thread | 🧵 Generative AI for Devs: LLMs, Diffusion, Transformers Explained! | Quick dive into the     │
│  core of Generative AI! What are LLMs, Diffusion Models, and Transformers? Why should developers care? #GenAI   │
│  #Developers #AIExplained | Pillar 1 |                                                                          │
│  | **Day 3** | Blog | Blog Post | GitHub Copilot vs. GitLab Duo: Choosing Your AI Code Assistant | Ready to     │
│  supercharge your coding with AI? Let's compare the top assistants and find your perfect co-pilot. | Pillar 2   │
│  |                                                                                                              │
│  | **Day 4** | YouTube | Tutorial Video | Boost Productivity: AI for Code Generation & Boilerplate Creation     │
│  (Live Demo) | Tired of writing repetitive code? Let AI handle the boilerplate so you can focus on innovation.  │
│  Watch how! | Pillar 3 |                                                                                        │
│  | | X | Poll/Question | Which AI Code Assistant do you use? Copilot, Duo, or something else? | Supercharge     │
│  your coding! Which AI code assistant is your go-to? Share your thoughts and why! #AICode #DevTools             │
│  #GitHubCopilot #GitLabDuo | Pillar 2 |                                                                         │
│  | **Day 5** | Blog | Blog Post | Prompt Engineering for Developers: Crafting Effective Code Prompts | The      │
│  secret to great AI-generated code isn't just the model, it's how you ask. | Pillar 4 |                         │
│  | **Day 6** | Blog | Blog Post | Tokens, Embeddings, At